In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import TopKCategoricalAccuracy


2025-06-24 10:49:39.196635: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750762179.430232      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750762179.493943      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Step 1: Load data
to_read = pd.read_csv("/kaggle/input/goodbooks-10k/to_read.csv")

# Step 2: Build user reading sequences
user_sequences = defaultdict(list)
for row in to_read.itertuples(index=False):
    user_sequences[row.user_id].append(str(row.book_id))  # Use str for tokenizer compatibility

# Step 3: Prepare input-output pairs
sequences = []
sequence_length = 5  # N previous books → predict next book

for user, books in user_sequences.items():
    if len(books) > sequence_length:
        for i in range(sequence_length, len(books)):
            seq = books[i-sequence_length:i+1]  # input + target
            sequences.append(seq)

print(f"Total training sequences: {len(sequences)}")


Total training sequences: 693648


In [ ]:
# Step 4: Tokenize book IDs
tokenizer = Tokenizer()
tokenizer.fit_on_texts([seq for seq in sequences])  # learn from all sequences

# Convert book IDs to integers
encoded_sequences = tokenizer.texts_to_sequences(sequences)

# Ensure all sequences are of same length
encoded_sequences = np.array(pad_sequences(encoded_sequences, maxlen=sequence_length + 1, padding='pre'))

# Split into X and y
X, y = encoded_sequences[:, :-1], encoded_sequences[:, -1]

# Total number of unique books (vocabulary size)
vocab_size = len(tokenizer.word_index) + 1
print(f"Vocabulary Size (unique books): {vocab_size}")


Vocabulary Size (unique books): 9987


In [ ]:
# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy

# Define model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=X_train.shape[1]),
    LSTM(128, return_sequences=False),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

# Compile model with top-5 accuracy
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=[
        'accuracy',
        SparseTopKCategoricalAccuracy(k=5, name='top_5_accuracy')
    ]
)

# Early stopping to avoid overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=128,
    validation_data=(X_val, y_val),
    callbacks=[early_stop]
)

Epoch 1/20
4878/4878 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - accuracy: 0.0063 - loss: 7.7184 - top_5_accuracy: 0.0258 - val_accuracy: 0.0229 - val_loss: 6.5712 - val_top_5_accuracy: 0.0814
Epoch 2/20
4878/4878 ━━━━━━━━━━━━━━━━━━━━ 37s 8ms/step - accuracy: 0.0247 - loss: 6.4308 - top_5_accuracy: 0.0869 - val_accuracy: 0.0335 - val_loss: 6.1949 - val_top_5_accuracy: 0.1139
Epoch 3/20
4878/4878 ━━━━━━━━━━━━━━━━━━━━ 37s 8ms/step - accuracy: 0.0356 - loss: 6.0432 - top_5_accuracy: 0.1180 - val_accuracy: 0.0436 - val_loss: 6.0331 - val_top_5_accuracy: 0.1346
Epoch 4/20
4878/4878 ━━━━━━━━━━━━━━━━━━━━ 37s 8ms/step - accuracy: 0.0449 - loss: 5.8321 - top_5_accuracy: 0.1386 - val_accuracy: 0.0506 - val_loss: 5.9486 - val_top_5_accuracy: 0.1477
Epoch 5/20
4878/4878 ━━━━━━━━━━━━━━━━━━━━ 37s 8ms/step - accuracy: 0.0516 - loss: 5.6877 - top_5_accuracy: 0.1530 - val_accuracy: 0.0535 - val_loss: 5.9070 - val_top_5_accuracy: 0.1545
Epoch 6/20
4878/4878 ━━━━━━━━━━━━━━━━━━━━ 37s 8ms/step - accuracy: 0.0574 -

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Concatenate


In [ ]:
# Load necessary CSVs
ratings = pd.read_csv('/kaggle/input/goodbooks-10k/ratings.csv')
books = pd.read_csv('/kaggle/input/goodbooks-10k/books.csv')
book_tags = pd.read_csv('/kaggle/input/goodbooks-10k/book_tags.csv')


In [ ]:
# Sort ratings by user and timestamp-like order (if exists)
ratings = ratings.sort_values(by=['user_id', 'book_id'])  # (no timestamp column)

# Group by user and extract sequences of at least 6 books
user_sequences = ratings.groupby('user_id')['book_id'].apply(list)
sequences = [seq for seq in user_sequences if len(seq) >= 6]


In [ ]:
X, y = [], []

for seq in sequences:
    for i in range(len(seq) - 5):
        X.append(seq[i:i+5])
        y.append(seq[i+5])

X = np.array(X)
y = np.array(y)

print("Total sequences:", len(X))


Total sequences: 754390


In [ ]:
# Extract useful metadata
book_meta = books[['book_id', 'average_rating', 'original_publication_year']].copy()
book_meta.fillna(0, inplace=True)

# Create book_id → [rating, year] dictionary
book_meta_dict = book_meta.set_index('book_id')[['average_rating', 'original_publication_year']].to_dict('index')

#  Fixed: Safely extract values
def build_meta_sequence(book_ids):
    return [
        list(book_meta_dict[bid].values()) if bid in book_meta_dict else [0, 0]
        for bid in book_ids
    ]

# Build metadata sequences
meta_X = np.array([build_meta_sequence(seq) for seq in X])


In [ ]:
# Map book_ids to integers (tokenize)
from sklearn.preprocessing import LabelEncoder

book_encoder = LabelEncoder()
all_book_ids = np.concatenate((X.flatten(), y))
book_encoder.fit(all_book_ids)

X_enc = book_encoder.transform(X.flatten()).reshape(X.shape)
y_enc = book_encoder.transform(y)

vocab_size = len(book_encoder.classes_)
print("Vocabulary size (books):", vocab_size)


Vocabulary size (books): 10000


In [ ]:
X_train, X_val, meta_train, meta_val, y_train, y_val = train_test_split(
    X_enc, meta_X, y_enc, test_size=0.1, random_state=42
)


In [ ]:
# Book sequence input
seq_input = Input(shape=(5,), name='book_sequence_input')
seq_embed = Embedding(input_dim=vocab_size, output_dim=128)(seq_input)
seq_lstm = LSTM(128)(seq_embed)

# Metadata input
meta_input = Input(shape=(5, 2), name='metadata_input')
meta_lstm = LSTM(32)(meta_input)

# Combine both branches
merged = Concatenate()([seq_lstm, meta_lstm])
dense = Dense(128, activation='relu')(merged)
drop = Dropout(0.3)(dense)
output = Dense(vocab_size, activation='softmax')(drop)

# Define model
model = Model(inputs=[seq_input, meta_input], outputs=output)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)]
)

model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ book_sequence_input       │ (None, 5)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 5, 128)         │      1,280,000 │ book_sequence_input[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ metadata_input            │ (None, 5, 2)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ (None, 128)            │        131,584 │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_4 (LSTM)             │ (None, 32)             │          4,480 │ metadata_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 160)            │              0 │ lstm_3[0][0],          │
│                           │                        │                │ lstm_4[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 128)            │         20,608 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 128)            │              0 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 10000)          │      1,290,000 │ dropout_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,726,672 (10.40 MB)

 Trainable params: 2,726,672 (10.40 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define EarlyStopping callback
early_stop = EarlyStopping(
    monitor='val_loss',       # Monitor validation loss
    patience=3,               # Stop if no improvement for 3 epochs
    restore_best_weights=True, # Restore the best weights seen during training
    verbose=1
)

# Train the model with early stopping
history = model.fit(
    [X_train, meta_train],    # Multi-input: sequences + metadata
    y_train,
    validation_data=([X_val, meta_val], y_val),
    epochs=20,
    batch_size=128,
    callbacks=[early_stop]    # Attach the callback here
)


Epoch 1/20
5305/5305 ━━━━━━━━━━━━━━━━━━━━ 44s 8ms/step - accuracy: 0.2682 - loss: 3.3785 - sparse_top_k_categorical_accuracy: 0.5257 - val_accuracy: 0.1325 - val_loss: 5.7145 - val_sparse_top_k_categorical_accuracy: 0.2823
Epoch 2/20
5305/5305 ━━━━━━━━━━━━━━━━━━━━ 44s 8ms/step - accuracy: 0.2725 - loss: 3.3443 - sparse_top_k_categorical_accuracy: 0.5312 - val_accuracy: 0.1312 - val_loss: 5.7409 - val_sparse_top_k_categorical_accuracy: 0.2818
Epoch 3/20
5305/5305 ━━━━━━━━━━━━━━━━━━━━ 44s 8ms/step - accuracy: 0.2769 - loss: 3.3113 - sparse_top_k_categorical_accuracy: 0.5370 - val_accuracy: 0.1310 - val_loss: 5.7967 - val_sparse_top_k_categorical_accuracy: 0.2801
Epoch 4/20
5305/5305 ━━━━━━━━━━━━━━━━━━━━ 44s 8ms/step - accuracy: 0.2817 - loss: 3.2747 - sparse_top_k_categorical_accuracy: 0.5444 - val_accuracy: 0.1300 - val_loss: 5.8803 - val_sparse_top_k_categorical_accuracy: 0.2794
Epoch 4: early stopping
Restoring model weights from the end of the best epoch: 1.


In [ ]:
# ===============================
#  1. IMPORT REQUIRED LIBRARIES
# ===============================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

In [ ]:
# =====================
# 2. LOAD THE DATA
# =====================
to_read = pd.read_csv('/kaggle/input/goodbooks-10k/to_read.csv')
books = pd.read_csv('/kaggle/input/goodbooks-10k/books.csv')

In [ ]:
# =====================================
# 3. GROUP BOOK SEQUENCES BY USER
# =====================================
user_sequences = to_read.groupby('user_id')['book_id'].apply(list)
user_sequences = user_sequences[user_sequences.apply(len) > 5]  # filter users with short history

In [ ]:
# ====================================
# 4. LABEL ENCODE BOOK_IDs
# ====================================
book_encoder = LabelEncoder()
book_encoder.fit(books['book_id'])

# Map books to encoded integers
encoded_sequences = []
targets = []

for sequence in user_sequences:
    encoded = book_encoder.transform([book for book in sequence if book in book_encoder.classes_])
    for i in range(1, len(encoded)):
        encoded_sequences.append(encoded[:i])
        targets.append(encoded[i])

In [ ]:
# ==========================================
# 5. PAD SEQUENCES AND PREPARE DATASETS
# ==========================================
max_seq_len = max(len(seq) for seq in encoded_sequences)
X = pad_sequences(encoded_sequences, maxlen=max_seq_len, padding='pre')
y = np.array(targets)

In [ ]:
# ================================
# 6. TRAIN-VALIDATION SPLIT
# ================================
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
# ========================================
# 7. DEFINE 2-LAYER LSTM MODEL
# ========================================
vocab_size = len(book_encoder.classes_) + 1  # +1 for padding token
embedding_dim = 64
lstm_units_1 = 128
lstm_units_2 = 64
dropout_rate = 0.3

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_seq_len),
    LSTM(lstm_units_1, return_sequences=True),
    Dropout(dropout_rate),
    LSTM(lstm_units_2),
    Dropout(dropout_rate),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# ========================================
# 8. EARLY STOPPING CALLBACK
# ========================================
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
# ===============================
# 9. TRAIN THE MODEL
# ===============================
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=256,
    callbacks=[early_stopping],
    verbose=2
)

Epoch 1/20
191/191 - 6s - 32ms/step - accuracy: 0.0123 - loss: 6.4432 - val_accuracy: 0.0070 - val_loss: 6.0624
Epoch 2/20
191/191 - 2s - 10ms/step - accuracy: 0.0144 - loss: 5.9302 - val_accuracy: 0.0223 - val_loss: 5.7446
Epoch 3/20
191/191 - 2s - 10ms/step - accuracy: 0.0240 - loss: 5.6959 - val_accuracy: 0.0287 - val_loss: 5.5966
Epoch 4/20
191/191 - 2s - 10ms/step - accuracy: 0.0290 - loss: 5.5789 - val_accuracy: 0.0328 - val_loss: 5.5075
Epoch 5/20
191/191 - 2s - 10ms/step - accuracy: 0.0327 - loss: 5.5021 - val_accuracy: 0.0392 - val_loss: 5.4580
Epoch 6/20
191/191 - 2s - 11ms/step - accuracy: 0.0345 - loss: 5.4574 - val_accuracy: 0.0429 - val_loss: 5.4133
Epoch 7/20
191/191 - 2s - 10ms/step - accuracy: 0.0369 - loss: 5.4149 - val_accuracy: 0.0424 - val_loss: 5.3757
Epoch 8/20
191/191 - 2s - 11ms/step - accuracy: 0.0407 - loss: 5.3828 - val_accuracy: 0.0459 - val_loss: 5.3506
Epoch 9/20
191/191 - 2s - 11ms/step - accuracy: 0.0420 - loss: 5.3604 - val_accuracy: 0.0446 - val_loss:

In [ ]:
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Accuracy: {val_acc:.4f}")


Validation Accuracy: 0.0490


In [ ]:
def recommend_books_for_user(user_id, top_k=5):
    if user_id not in user_sequences:
        return "No data for this user."

    last_sequence = user_sequences[user_id]
    encoded_seq = book_encoder.transform([b for b in last_sequence if b in book_encoder.classes_])
    padded_seq = pad_sequences([encoded_seq], maxlen=max_seq_len, padding='pre')

    prediction = model.predict(padded_seq, verbose=0)[0]
    top_indices = prediction.argsort()[-top_k:][::-1]
    predicted_book_ids = book_encoder.inverse_transform(top_indices)

    return books[books['book_id'].isin(predicted_book_ids)][['book_id', 'title', 'authors']]

# Example:
recommend_books_for_user(1)


,book_id,title,authors
292,295,Treasure Island,Robert Louis Stevenson
1998,249,Tropic of Cancer,Henry Miller
5071,359,"The Salmon of Doubt (Dirk Gently, #3)",Douglas Adams
7241,291,The Broken Wings,"Kahlil Gibran, Anthony R. Ferris"
7903,355,Job: A Comedy of Justice,Robert A. Heinlein


In [ ]:
# ===================== 📦 LOAD LIBRARIES =====================
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import TopKCategoricalAccuracy

# ===================== 📁 LOAD DATA =====================
ratings = pd.read_csv('/kaggle/input/goodbooks-10k/ratings.csv')
to_read = pd.read_csv('/kaggle/input/goodbooks-10k/to_read.csv')
books = pd.read_csv('/kaggle/input/goodbooks-10k/books.csv')

# ===================== 📊 FILTER POPULAR BOOKS =====================
popular_books = to_read['book_id'].value_counts()
popular_books = popular_books[popular_books >= 20].index.tolist()
ratings = ratings[ratings['book_id'].isin(popular_books)]

# ===================== 🔢 ENCODE USER & BOOK IDs =====================
ratings = ratings.sort_values(['user_id', 'book_id'])

user_enc = LabelEncoder()
book_enc = LabelEncoder()

ratings['user'] = user_enc.fit_transform(ratings['user_id'])
ratings['book'] = book_enc.fit_transform(ratings['book_id'])

n_users = ratings['user'].nunique()
n_books = ratings['book'].nunique()

In [ ]:
# ===================== 📚 CREATE SEQUENCES =====================
def create_sequences(df, sequence_length=5):
    X, y = [], []
    for user in df['user'].unique():
        user_books = df[df['user'] == user]['book'].tolist()
        if len(user_books) > sequence_length:
            for i in range(len(user_books) - sequence_length):
                X.append(user_books[i:i+sequence_length])
                y.append(user_books[i+sequence_length])
    return np.array(X), np.array(y)

X, y = create_sequences(ratings)

# ===================== 🧠 ADD METADATA =====================
book_meta = books[['book_id', 'average_rating', 'ratings_count', 'language_code']]
book_meta = book_meta[book_meta['book_id'].isin(book_enc.inverse_transform(range(n_books)))]
book_meta['book'] = book_enc.transform(book_meta['book_id'])

book_features = book_meta.set_index('book')[['average_rating', 'ratings_count']].fillna(0)
book_features['ratings_count'] = np.log1p(book_features['ratings_count'])

book_feature_matrix = np.zeros((n_books, 2))
for book_id in book_features.index:
    book_feature_matrix[book_id] = book_features.loc[book_id].values

X_metadata = np.array([[book_feature_matrix[bid] for bid in seq] for seq in X])

In [ ]:
# ===================== 🧪 TRAIN / VAL SPLIT =====================
X_train, X_val, Xmeta_train, Xmeta_val, y_train, y_val = train_test_split(
    X, X_metadata, y, test_size=0.1, random_state=42
)

# ===================== 🤖 MODEL BUILDING =====================
embedding_dim = 64
meta_dim = 2
seq_len = X.shape[1]  # should be 5
# n_books already defined

# Inputs
book_input = Input(shape=(seq_len,), name='book_input')
meta_input = Input(shape=(seq_len, meta_dim), name='meta_input')

# Embedding + metadata concat
book_embedding = Embedding(input_dim=n_books, output_dim=embedding_dim)(book_input)
merged = Concatenate()([book_embedding, meta_input])

# LSTM layers
x = LSTM(128, return_sequences=True)(merged)
x = LSTM(64)(x)
x = Dropout(0.3)(x)

# Output
output = Dense(n_books, activation='softmax')(x)

from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy  # ✅ Add this

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=[SparseTopKCategoricalAccuracy(k=5, name='top_5_accuracy')]  # ✅ Fix
)


model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ book_input (InputLayer)   │ (None, 5)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 5, 64)          │        483,264 │ book_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ meta_input (InputLayer)   │ (None, 5, 2)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 5, 66)          │              0 │ embedding_1[0][0],     │
│ (Concatenate)             │                        │                │ meta_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ (None, 5, 128)         │         99,840 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ (None, 64)             │         49,408 │ lstm_2[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 64)             │              0 │ lstm_3[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 7551)           │        490,815 │ dropout_1[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,123,327 (4.29 MB)

 Trainable params: 1,123,327 (4.29 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ===================== 🏋️‍♂️ TRAINING =====================
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(
    [X_train, Xmeta_train], y_train,
    validation_data=([X_val, Xmeta_val], y_val),
    batch_size=128,
    epochs=20,
    callbacks=[early_stop]
)

Epoch 1/20


I0000 00:00:1750827553.275279     104 cuda_dnn.cc:529] Loaded cuDNN version 90300


3935/3935 ━━━━━━━━━━━━━━━━━━━━ 30s 6ms/step - loss: 8.1384 - top_5_accuracy: 0.0087 - val_loss: 6.4408 - val_top_5_accuracy: 0.0486
Epoch 2/20
3935/3935 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 6.2930 - top_5_accuracy: 0.0579 - val_loss: 5.7712 - val_top_5_accuracy: 0.1025
Epoch 3/20
3935/3935 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 5.7586 - top_5_accuracy: 0.1056 - val_loss: 5.4785 - val_top_5_accuracy: 0.1412
Epoch 4/20
3935/3935 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 5.4984 - top_5_accuracy: 0.1398 - val_loss: 5.3035 - val_top_5_accuracy: 0.1724
Epoch 5/20
3935/3935 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 5.3254 - top_5_accuracy: 0.1682 - val_loss: 5.1682 - val_top_5_accuracy: 0.1958
Epoch 6/20
3935/3935 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 5.1878 - top_5_accuracy: 0.1915 - val_loss: 5.0776 - val_top_5_accuracy: 0.2110
Epoch 7/20
3935/3935 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 5.0805 - top_5_accuracy: 0.2102 - val_loss: 4.9989 - val_top_5_accuracy: 0.2242
Epoch 8/20

In [ ]:
val_loss, val_top5 = model.evaluate([X_val, Xmeta_val], y_val)
print(f"✅ Validation Top-5 Accuracy: {val_top5:.4f}")


1749/1749 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 4.8311 - top_5_accuracy: 0.2668
✅ Validation Top-5 Accuracy: 0.2697


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import TopKCategoricalAccuracy

# ========== Load Data ==========
ratings = pd.read_csv('/kaggle/input/goodbooks-10k/ratings.csv')
to_read = pd.read_csv('/kaggle/input/goodbooks-10k/to_read.csv')
books = pd.read_csv('/kaggle/input/goodbooks-10k/books.csv')

# ========== Filter Rare Books ==========
popular_books = to_read['book_id'].value_counts()
popular_books = popular_books[popular_books >= 20].index.tolist()
ratings = ratings[ratings['book_id'].isin(popular_books)]

# ========== Sort and Encode ==========
ratings = ratings.sort_values(['user_id', 'book_id'])
user_enc = LabelEncoder()
book_enc = LabelEncoder()

ratings['user'] = user_enc.fit_transform(ratings['user_id'])
ratings['book'] = book_enc.fit_transform(ratings['book_id'])

n_users = ratings['user'].nunique()
n_books = ratings['book'].nunique()

# ========== Build Sequences ==========
def create_sequences(df, sequence_length=5):
    X, y = [], []
    for user in df['user'].unique():
        user_data = df[df['user'] == user]['book'].tolist()
        if len(user_data) > sequence_length:
            for i in range(len(user_data) - sequence_length):
                X.append(user_data[i:i+sequence_length])
                y.append(user_data[i+sequence_length])
    return np.array(X), np.array(y)

X, y = create_sequences(ratings)

# ========== Prepare Metadata ==========
book_meta = books[['book_id', 'average_rating', 'ratings_count']]
book_meta = book_meta[book_meta['book_id'].isin(book_enc.inverse_transform(range(n_books)))]
book_meta['book'] = book_enc.transform(book_meta['book_id'])

# Normalize metadata
book_features = book_meta.set_index('book')[['average_rating', 'ratings_count']].fillna(0)
book_features['ratings_count'] = np.log1p(book_features['ratings_count'])

# Build feature matrix
book_feature_matrix = np.zeros((n_books, 2), dtype=np.float32)
for book_id in book_features.index:
    book_feature_matrix[book_id] = book_features.loc[book_id].values

# Map metadata to each sequence
X_metadata = np.array([
    [book_feature_matrix[book_id] for book_id in seq]
    for seq in X
], dtype=np.float32)

# ========== Train/Validation Split ==========
X_train, X_val, Xmeta_train, Xmeta_val, y_train, y_val = train_test_split(
    X, X_metadata, y, test_size=0.1, random_state=42
)

# ========== Build GRU Model ==========
embedding_dim = 64
meta_dim = 2
seq_len = X.shape[1]

book_input = Input(shape=(seq_len,), name='book_input')
meta_input = Input(shape=(seq_len, meta_dim), name='meta_input')

# Embedding + metadata concatenation
book_embedding = Embedding(input_dim=n_books, output_dim=embedding_dim)(book_input)
merged = Concatenate()([book_embedding, meta_input])

# GRU layers
x = GRU(128, return_sequences=True)(merged)
x = GRU(64)(x)
x = Dropout(0.3)(x)

# Output layer
output = Dense(n_books, activation='softmax')(x)

from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy  # ✅ Add this

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=[SparseTopKCategoricalAccuracy(k=5, name='top_5_accuracy')]  # ✅ Fix
)

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ book_input (InputLayer)   │ (None, 5)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_4 (Embedding)   │ (None, 5, 64)          │        483,264 │ book_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ meta_input (InputLayer)   │ (None, 5, 2)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_4             │ (None, 5, 66)          │              0 │ embedding_4[0][0],     │
│ (Concatenate)             │                        │                │ meta_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_2 (GRU)               │ (None, 5, 128)         │         75,264 │ concatenate_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_3 (GRU)               │ (None, 64)             │         37,248 │ gru_2[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 64)             │              0 │ gru_3[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 7551)           │        490,815 │ dropout_4[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,086,591 (4.15 MB)

 Trainable params: 1,086,591 (4.15 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ========== Train Model ==========
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(
    [X_train, Xmeta_train], y_train,
    validation_data=([X_val, Xmeta_val], y_val),
    batch_size=128,
    epochs=20,
    callbacks=[early_stop]
)

Epoch 1/20
3935/3935 ━━━━━━━━━━━━━━━━━━━━ 27s 6ms/step - loss: 8.0917 - top_5_accuracy: 0.0103 - val_loss: 6.4016 - val_top_5_accuracy: 0.0530
Epoch 2/20
3935/3935 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 6.2707 - top_5_accuracy: 0.0619 - val_loss: 5.7851 - val_top_5_accuracy: 0.1038
Epoch 3/20
3935/3935 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 5.7852 - top_5_accuracy: 0.1033 - val_loss: 5.5056 - val_top_5_accuracy: 0.1373
Epoch 4/20
3935/3935 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 5.5364 - top_5_accuracy: 0.1356 - val_loss: 5.3220 - val_top_5_accuracy: 0.1680
Epoch 5/20
3935/3935 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 5.3584 - top_5_accuracy: 0.1637 - val_loss: 5.1980 - val_top_5_accuracy: 0.1894
Epoch 6/20
3935/3935 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 5.2282 - top_5_accuracy: 0.1875 - val_loss: 5.1120 - val_top_5_accuracy: 0.2073
Epoch 7/20
3935/3935 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 5.1296 - top_5_accuracy: 0.2039 - val_loss: 5.0475 - val_top_5_accuracy: 0.2192

In [ ]:
# Evaluate the model on the validation set
val_loss, val_top5_acc = model.evaluate([X_val, Xmeta_val], y_val, batch_size=128)
print(f"\n✅ Validation Top-5 Accuracy: {val_top5_acc:.4f}")


438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 4.8677 - top_5_accuracy: 0.2616

✅ Validation Top-5 Accuracy: 0.2650


In [ ]:
# ========== Imports ==========
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import TopKCategoricalAccuracy

# ========== Load Data ==========
ratings = pd.read_csv('/kaggle/input/goodbooks-10k/ratings.csv')
to_read = pd.read_csv('/kaggle/input/goodbooks-10k/to_read.csv')
books = pd.read_csv('/kaggle/input/goodbooks-10k/books.csv')

# ========== Filter Rare Books ==========
popular_books = to_read['book_id'].value_counts()
popular_books = popular_books[popular_books >= 20].index.tolist()
ratings = ratings[ratings['book_id'].isin(popular_books)]

# ========== Sort and Encode ==========
ratings = ratings.sort_values(['user_id', 'book_id'])
user_enc = LabelEncoder()
book_enc = LabelEncoder()

ratings['user'] = user_enc.fit_transform(ratings['user_id'])
ratings['book'] = book_enc.fit_transform(ratings['book_id'])

n_users = ratings['user'].nunique()
n_books = ratings['book'].nunique()

# ========== Build Sequences ==========
def create_sequences(df, sequence_length=10):
    X, y = [], []
    for user in df['user'].unique():
        user_data = df[df['user'] == user]['book'].tolist()
        if len(user_data) > sequence_length:
            for i in range(len(user_data) - sequence_length):
                X.append(user_data[i:i+sequence_length])
                y.append(user_data[i+sequence_length])
    return np.array(X), np.array(y)

X, y = create_sequences(ratings)

# ========== Prepare Metadata ==========
book_meta = books[['book_id', 'average_rating', 'ratings_count']]
book_meta = book_meta[book_meta['book_id'].isin(book_enc.inverse_transform(range(n_books)))]
book_meta['book'] = book_enc.transform(book_meta['book_id'])

# Normalize metadata
book_features = book_meta.set_index('book')[['average_rating', 'ratings_count']].fillna(0)
book_features['ratings_count'] = np.log1p(book_features['ratings_count'])

# Build feature matrix
book_feature_matrix = np.zeros((n_books, 2))
for book_id in book_features.index:
    book_feature_matrix[book_id] = book_features.loc[book_id].values

# Map metadata to each sequence
X_metadata = np.array([[book_feature_matrix[bid] for bid in seq] for seq in X])

# ========== Train/Validation Split ==========
X_train, X_val, Xmeta_train, Xmeta_val, y_train, y_val = train_test_split(
    X, X_metadata, y, test_size=0.1, random_state=42
)

# ========== Build GRU Model ==========
embedding_dim = 64
meta_dim = 2
seq_len = X.shape[1]

book_input = Input(shape=(seq_len,), name='book_input')
meta_input = Input(shape=(seq_len, meta_dim), name='meta_input')

book_embedding = Embedding(input_dim=n_books, output_dim=embedding_dim)(book_input)
merged = Concatenate()([book_embedding, meta_input])

x = GRU(128, return_sequences=True)(merged)
x = GRU(64)(x)
x = Dropout(0.3)(x)

output = Dense(n_books, activation='softmax')(x)

model = Model(inputs=[book_input, meta_input], outputs=output)
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy  # ✅ Add this

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=[SparseTopKCategoricalAccuracy(k=5, name='top_5_accuracy')]  # ✅ Fix
)
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ book_input (InputLayer)   │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_6 (Embedding)   │ (None, 10, 64)         │        483,264 │ book_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ meta_input (InputLayer)   │ (None, 10, 2)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_6             │ (None, 10, 66)         │              0 │ embedding_6[0][0],     │
│ (Concatenate)             │                        │                │ meta_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_6 (GRU)               │ (None, 10, 128)        │         75,264 │ concatenate_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_7 (GRU)               │ (None, 64)             │         37,248 │ gru_6[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_6 (Dropout)       │ (None, 64)             │              0 │ gru_7[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 7551)           │        490,815 │ dropout_6[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,086,591 (4.15 MB)

 Trainable params: 1,086,591 (4.15 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ========== Train ==========
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(
    [X_train, Xmeta_train], y_train,
    validation_data=([X_val, Xmeta_val], y_val),
    batch_size=128,
    epochs=20,
    callbacks=[early_stop]
)

Epoch 1/20
3164/3164 ━━━━━━━━━━━━━━━━━━━━ 23s 6ms/step - loss: 8.1245 - top_5_accuracy: 0.0105 - val_loss: 6.4575 - val_top_5_accuracy: 0.0509
Epoch 2/20
3164/3164 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 6.3008 - top_5_accuracy: 0.0598 - val_loss: 5.7691 - val_top_5_accuracy: 0.1033
Epoch 3/20
3164/3164 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 5.7456 - top_5_accuracy: 0.1039 - val_loss: 5.4228 - val_top_5_accuracy: 0.1511
Epoch 4/20
3164/3164 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 5.4371 - top_5_accuracy: 0.1456 - val_loss: 5.2098 - val_top_5_accuracy: 0.1827
Epoch 5/20
3164/3164 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 5.2314 - top_5_accuracy: 0.1780 - val_loss: 5.0614 - val_top_5_accuracy: 0.2108
Epoch 6/20
3164/3164 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 5.0734 - top_5_accuracy: 0.2037 - val_loss: 4.9563 - val_top_5_accuracy: 0.2293
Epoch 7/20
3164/3164 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 4.9521 - top_5_accuracy: 0.2264 - val_loss: 4.8894 - val_top_5_accuracy: 0.2391

In [ ]:
# Evaluate on validation data
val_loss, val_top5 = model.evaluate([X_val, Xmeta_val], y_val, batch_size=128)

# Print formatted result
print(f"\n✅ Validation Top-5 Accuracy: {val_top5:.4f}")


352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 4.6748 - top_5_accuracy: 0.2923

✅ Validation Top-5 Accuracy: 0.2908
